In [117]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler  # Changed from StandardScaler to MinMaxScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report,
    roc_curve, precision_recall_curve
)
import joblib
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [118]:
# Load data
df = pd.read_csv("../data/synthetic/features_with_pdlabel.csv")

# Define features
pd_features = [
    "avgMonthlyIncome",
    "emiRatio",
    "expenseRatio",
    "bounceCount",
    "accountAgeMonths"
]

X = df[pd_features].copy()
y = df["pdLabel"].copy()

print(f"📊 Class Distribution:")
print(f"   Non-Default (0): {(y == 0).sum():,} ({100 * (y == 0).sum() / len(y):.2f}%)")
print(f"   Default (1):     {(y == 1).sum():,} ({100 * (y == 1).sum() / len(y):.2f}%)")

📊 Class Distribution:
   Non-Default (0): 20,783 (69.28%)
   Default (1):     9,217 (30.72%)


In [119]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print("TRAIN/TEST SPLIT")
print(f"\n📊 Training Set: {len(y_train):,} samples")
print(f"   Non-Default: {(y_train == 0).sum():,} ({100 * (y_train == 0).sum() / len(y_train):.2f}%)")
print(f"   Default:     {(y_train == 1).sum():,} ({100 * (y_train == 1).sum() / len(y_train):.2f}%)")
print()
print(f"📊 Test Set: {len(y_test):,} samples")
print(f"   Non-Default: {(y_test == 0).sum():,} ({100 * (y_test == 0).sum() / len(y_test):.2f}%)")
print(f"   Default:     {(y_test == 1).sum():,} ({100 * (y_test == 1).sum() / len(y_test):.2f}%)")

TRAIN/TEST SPLIT

📊 Training Set: 24,000 samples
   Non-Default: 16,626 (69.28%)
   Default:     7,374 (30.73%)

📊 Test Set: 6,000 samples
   Non-Default: 4,157 (69.28%)
   Default:     1,843 (30.72%)


In [120]:
# Scale features using MinMaxScaler (most numerically stable)
print("Scaling features (MinMaxScaler)...")
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Scaling complete")
print(f"   Train range: [{X_train_scaled.min():.4f}, {X_train_scaled.max():.4f}]")
print(f"   Test range:  [{X_test_scaled.min():.4f}, {X_test_scaled.max():.4f}]")

# Train model with robust parameters
print("\nTraining model...")
pd_model = LogisticRegression(
    max_iter=500,
    class_weight="balanced",
    solver="saga",
    C=0.5,
    random_state=42,
    tol=1e-3,
    n_jobs=-1
)

pd_model.fit(X_train_scaled, y_train)

Scaling features (MinMaxScaler)...
Scaling complete
   Train range: [0.0000, 1.0000]
   Test range:  [0.0000, 1.0051]

Training model...


LogisticRegression(C=0.5, class_weight='balanced', max_iter=500, n_jobs=-1,
                   random_state=42, solver='saga', tol=0.001)

In [121]:
# Generate predictions
y_pred = pd_model.predict(X_test_scaled)
y_pred_proba = pd_model.predict_proba(X_test_scaled)[:, 1]

print("Predictions generated successfully")

Predictions generated successfully


In [122]:
print("📈 MODEL PERFORMANCE METRICS")


print(f"\nAccuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, y_pred_proba):.4f}")

print("\n📊 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(f"   TN: {cm[0,0]:<6} FP: {cm[0,1]:<6}")
print(f"   FN: {cm[1,0]:<6} TP: {cm[1,1]:<6}")

📈 MODEL PERFORMANCE METRICS

Accuracy:  0.8895
Precision: 0.7726
Recall:    0.9072
F1 Score:  0.8345
ROC AUC:   0.9624

📊 Confusion Matrix:
   TN: 3665   FP: 492   
   FN: 171    TP: 1672  


In [123]:
# Generate PD predictions for the full dataset
print("🎯 Generating PD predictions for full dataset...")
X_full_scaled = scaler.transform(X)
df["PD"] = pd_model.predict_proba(X_full_scaled)[:, 1]

🎯 Generating PD predictions for full dataset...


In [124]:
# Save model artifacts
print("Saving model artifacts...")

joblib.dump(pd_model, '../models/pd_model.joblib')
joblib.dump(scaler, '../models/pd_scaler.joblib')
joblib.dump(pd_features, '../models/pd_features.joblib')

print("Model artifacts saved:")
print("   • pd_model.joblib")
print("   • pd_scaler.joblib")
print("   • pd_features.joblib")

# Save predictions to CSV
df.to_csv("../data/synthetic/features_with_pd_predictions.csv", index=False)

Saving model artifacts...
Model artifacts saved:
   • pd_model.joblib
   • pd_scaler.joblib
   • pd_features.joblib


In [133]:
# CUSTOM INPUT

# custom_input = {
#     "avgMonthlyIncome": 250000,      # Average monthly income in rupees
#     "emiRatio": 0.25,                # EMI to income ratio (0-1)
#     "expenseRatio": 0.35,            # Expense to income ratio (0-1)
#     "bounceCount": 0,                # Number of payment bounces
#     "accountAgeMonths": 60           # Account age in months
# }

custom_input = {
    "avgMonthlyIncome": 70000,
    "emiRatio": 0.30,
    "expenseRatio": 0.81,
    "bounceCount": 2,
    "accountAgeMonths": 18
}


# Create dataframe from custom input
custom_df = pd.DataFrame([custom_input])

print("\n📝 Input Features:")
for feature, value in custom_input.items():
    print(f"{feature:<25} {value:>12}")

# Scale the custom input
custom_scaled = scaler.transform(custom_df[pd_features])

# Get prediction
custom_pd = pd_model.predict_proba(custom_scaled)[:, 1][0]
custom_class = pd_model.predict(custom_scaled)[0]

print("\n📊 Prediction Results:")
print(f"   • Probability of Default (PD): {custom_pd:.4f} ({custom_pd*100:.2f}%)")
print(f"   • Classification: {'🔴 DEFAULT RISK' if custom_class == 1 else '🟢 NON-DEFAULT'}")


📝 Input Features:
avgMonthlyIncome                 70000
emiRatio                           0.3
expenseRatio                      0.81
bounceCount                          2
accountAgeMonths                    18

📊 Prediction Results:
   • Probability of Default (PD): 0.3929 (39.29%)
   • Classification: 🟢 NON-DEFAULT
